### Prerequisites Setup
All required dependencies are set up here. This supports a production pipeline approach where reproducibility and environment setup are clearly defined.

In [1]:
# First I import the necessary libraries
# standard
import os
import sys
import re
import time
import json
import pickle
import logging
import asyncio
import aiohttp
import unicodedata
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.error import HTTPError, URLError
import urllib.request
import urllib.parse

# third parties
import pandas as pd
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from rapidfuzz import process, fuzz

# Here I also install Geopy, which is used for geocoding.
!{sys.executable} -m pip install geopy==2.4.1


Defaulting to user installation because normal site-packages is not writeable


## Data Normalization and Metadata Preparation
In this section, I normalize event titles using string operations. The raw JSON data, located in the folder `Ibsenstage_raw`, is flattened using `pandas.json_normalize()` for easier manipulation.
I remove the `venuecountry` column, since the dataset is focused exclusively on performances in Norway. To standardize the naming of events, I build a canonical list of titles using the `worktitle` field, then construct regular expressions to match common title variants. Some well-known plays (i.e. `Et dukkehjem`) have additional hardcoded variant patterns (i.e. "Nora", "Casa di bambola", etc.).
The `eventname` field is then normalized by matching against these compiled regex patterns.

In [2]:
# 1) I open the json file
json_path = '../Ibsenstage_raw/IbsenStage_scrape.json'
if not os.path.isfile(json_path):
    raise FileNotFoundError(f'File not found: {json_path}')
print('Using source file:', json_path)

# 2) Ensure output folder exists
staged_dir = Path('.')
staged_dir.mkdir(exist_ok=True)
json_out = staged_dir / 'IbsenStage_normalized.json'

# 3) Load & flatten JSON
with open(json_path, 'r', encoding='utf-8') as f:
    root = json.load(f)
records = root.get('hits', root)
ibsen_df = pd.json_normalize(records, sep='_')
ibsen_df.head(5)

Using source file: ../Ibsenstage_raw/IbsenStage_scrape.json


,eventname,eventid,first_date,workid,worktitle,venueid,venuename,venuecountry
0,Hedda Gabler,85542,1983-11-12,8547.0,Hedda Gabler,14985,Honningsvåg kino,Norway
1,Hedda Gabler,85543,1983-11-14,8547.0,Hedda Gabler,14981,Vadsø kino,Norway
2,Hedda Gabler,85544,1983-11-15,8547.0,Hedda Gabler,14983,Miljøbygget,Norway
3,Gengangere,85550,1983-10-14,8542.0,Ghosts,12401,Telemark Teater,Norway
4,Gengangere,85607,1984-01-30,8542.0,Ghosts,12730,Kristiansand Teater,Norway


In [3]:
# 4) Remove unnecessary columns
if 'venuecountry' in ibsen_df.columns:
    ibsen_df = ibsen_df.drop(columns=['venuecountry'])

# 5) Build canonical patterns for normalization
unique_titles = (
    ibsen_df['worktitle']
    .dropna()
    .astype(str)
    .str.strip()
    .sort_values()
    .unique()
)
canonical = {}
for title in unique_titles:
    safe = re.escape(title).replace('\\\\ ', '[\\\\s_-]*')
    canonical[title] = ['^' + safe + '$']

extra_variants = {
    'Et dukkehjem'   : ['^a doll.*house$', '^ett[\\\\s_-]*dockhem$', '^casa[\\\\s_-]*di[\\\\s_-]*bambola$', '^nora$'],
    'Gjengangere'    : ['^ghosts$', '^spettri$'],
    'En folkefiende' : ['^an enemy.*people$'],
    'Vildanden'      : ['^the[\\\\s_-]*wild[\\\\s_-]*duck$'],
}
for canon, pats in extra_variants.items():
    canonical.setdefault(canon, []).extend(pats)

pattern_map = [
    (re.compile(pat, re.IGNORECASE), canon)
    for canon, pats in canonical.items()
    for pat in pats
]
def normalize_title(txt):
    if pd.isna(txt): return txt
    low = str(txt).strip().lower()
    for pat, canon in pattern_map:
        if pat.match(low): return canon
    return txt

# 6) Normalize eventname
ibsen_df['eventname'] = ibsen_df['eventname'].apply(normalize_title)

# 7) Save the normalized DataFrame to JSON
json_out = 'IbsenStage_normalized.json'
ibsen_df.to_json(json_out, orient='records', force_ascii=False, indent=2)
print('Saved normalized JSON →', json_out)

# 8) Preview
preview = json.loads(ibsen_df.head(3).to_json(orient='records', force_ascii=False))
print('Preview:')
print(json.dumps(preview, ensure_ascii=False, indent=2))

Saved normalized JSON → IbsenStage_normalized.json
Preview:
[
  {
    "eventname": "Hedda Gabler",
    "eventid": 85542,
    "first_date": "1983-11-12",
    "workid": 8547.0,
    "worktitle": "Hedda Gabler",
    "venueid": 14985,
    "venuename": "Honningsvåg kino"
  },
  {
    "eventname": "Hedda Gabler",
    "eventid": 85543,
    "first_date": "1983-11-14",
    "workid": 8547.0,
    "worktitle": "Hedda Gabler",
    "venueid": 14981,
    "venuename": "Vadsø kino"
  },
  {
    "eventname": "Hedda Gabler",
    "eventid": 85544,
    "first_date": "1983-11-15",
    "workid": 8547.0,
    "worktitle": "Hedda Gabler",
    "venueid": 14983,
    "venuename": "Miljøbygget"
  }
]


### Loading and Preparing the Dataset
The dataset is loaded and prepared with necessary libraries. This step ensures that I can apply transformations in a controlled and repeatable environment. I follow FAIR principles, especially focusing on reusability and interoperability. With this cell I give information about the cities connected to the venues, populating over 60% of the keys venuecity with actual city names thanks to GeoPy and GeoNames. 
To increase the accuracy, if the name of the city or a variation of it is included in the key `venuenames`, it will be mapped in the new key `venuecity` (i.e. "Teater i Trondheim" will give "Trondheim"). 

Here I also try to resolve some incongruencies, expecially related to Oslo, so that the key `venuecity` will connect back to that city in the instances of variation of Kristiania or Nationaltheatret. I also include some more common overrides to enhance the population. Ensuring the presence of `venuecity` is meaningful for a fallback when I will map `venueid` to URIs.

I initialize a persistent cache to store previously resolved venue-to-city mappings (`venue_geocode_cache.pkl`). A separate city list from the GeoNames API (limited to Norway) is also cached to avoid repeated calls. Unicode normalization is applied to venue names to reduce inconsistencies due to accents or formatting, and the resolved cities are added to the dataset. 

In [4]:
# Configuration
DATA_FOLDER     = "."
CACHE_PATH      = os.path.join(DATA_FOLDER, "venue_geocode_cache.pkl")
INPUT_FILE      = os.path.join(DATA_FOLDER, "IbsenStage_normalized.json")
OUTPUT_FILE     = os.path.join(DATA_FOLDER, "IbsenStage_with_city.json")
GEONAMES_USER   = "MBIB4140_ibsen_user"  # your GeoNames username
GEONAMES_COUNTRY = "NO"

# Nominatim Setup
geolocator = Nominatim(user_agent="ibsen_city_extractor", timeout=5)  # Reduced timeout
geocode    = RateLimiter(geolocator.geocode, min_delay_seconds=0.5, max_retries=1)  # Faster rate limiting

# Cache
if os.path.exists(CACHE_PATH):
    with open(CACHE_PATH, "rb") as f:
        cache = pickle.load(f)
else:
    cache = {}

# Norway cities list (cached)
def load_norway_cities():
    cache_file = os.path.join(DATA_FOLDER, "norway_cities_cache.pkl")
    if os.path.exists(cache_file):
        with open(cache_file, "rb") as f:
            return pickle.load(f)
    
    qs = urllib.parse.urlencode({
        "country": GEONAMES_COUNTRY,
        "featureClass": "P",
        "maxRows": 2000,
        "username": GEONAMES_USER
    })
    url = f"http://api.geonames.org/searchJSON?{qs}"
    try:
        with urllib.request.urlopen(url, timeout=10) as resp:
            data = json.load(resp)
            cities = {item["name"] for item in data.get("geonames", [])}
            # Cache the cities list
            with open(cache_file, "wb") as f:
                pickle.dump(cities, f)
            return cities
    except Exception:
        return set()

norway_cities = load_norway_cities()

# Print intermediate results
print("CONFIGURATION AND SETUP COMPLETE")
print(f"Data folder: {DATA_FOLDER} = IbsenStage_staged")
print(f"Cache path: {CACHE_PATH}")
print(f"Input file: {INPUT_FILE}")
print(f"Output file: {OUTPUT_FILE}")
print(f"GeoNames user: {GEONAMES_USER}")
print(f"Cache loaded: {len(cache)} entries")
print("Sample Norway cities:", list(norway_cities)[:10] if norway_cities else "None")

CONFIGURATION AND SETUP COMPLETE
Data folder: . = IbsenStage_staged
Cache path: .\venue_geocode_cache.pkl
Input file: .\IbsenStage_normalized.json
Output file: .\IbsenStage_with_city.json
GeoNames user: MBIB4140_ibsen_user
Cache loaded: 1302 entries
Sample Norway cities: None


In [5]:
# Precompute normalized overrides
OVERRIDES = {
    "nationaltheatret": "Oslo", "kristiania": "Oslo", "christiania": "Oslo",
    "det norske teatret": "Oslo", "black box teater": "Oslo", "oslo nye": "Oslo",
    "trøndelag teater": "Trondheim", "rosendal teater": "Trondheim",
    "den nationale scene": "Bergen", "dns": "Bergen", "hordaland teater": "Bergen",
    "kilden": "Kristiansand", "agder teater": "Kristiansand",
    "hålogaland teater": "Tromsø", "rogaland teater": "Stavanger",
    "teater innsikt": "Stavanger", "teater i drammen": "Drammen",
    "teater i fredrikstad": "Fredrikstad", "teater i moss": "Moss",
    "teater i ålesund": "Ålesund", "teater i bodø": "Bodø",
    "teater i tromsø": "Tromsø", "teater i sarpsborg": "Sarpsborg",
    "teater i skien": "Skien", "teater i hamar": "Hamar",
    "teater i sandnes": "Sandnes"
}

# Precompute normalized overrides for faster lookup
def normalize(txt):
    txt = unicodedata.normalize('NFKD', txt)
    txt = "".join(c for c in txt if not unicodedata.combining(c))
    return re.sub(r'[^a-z0-9]', '', txt.lower())

# Create normalized override mapping
NORMALIZED_OVERRIDES = {normalize(k): v for k, v in OVERRIDES.items()}
NORMALIZED_OVERRIDE_KEYS = list(NORMALIZED_OVERRIDES.keys())

# Constants
CITY_KEYS = [
    "city", "town", "village", "municipality", "hamlet",
    "locality", "county", "state_district", "state",
    "region", "district", "suburb"
]

# Optimized city extraction logic
def get_city_for(venue_name: str) -> str | None:
    if not venue_name or pd.isna(venue_name):
        return None
    name = venue_name.strip()
    
    # 1. Check cache first (done for optimization)
    if name in cache:
        return cache[name]
    
    key = name.lower()
    norm = normalize(key)

    # 2. Hard overrides (using precomputed normalized keys)
    for norm_key, city in NORMALIZED_OVERRIDES.items():
        if norm_key in norm:
            cache[name] = city
            return city

    # 3. Fuzzy match (only if no direct match found)
    if len(norm) > 3:  # Skip very short names for fuzzy matching
        match, score, _ = process.extractOne(norm, NORMALIZED_OVERRIDE_KEYS, scorer=fuzz.partial_ratio)
        if score > 85:  # Slightly lower threshold for better performance
            matched_city = NORMALIZED_OVERRIDES[match]
            cache[name] = matched_city
            return matched_city

    # 4. Token match against known cities (before expensive API calls)
    for token in re.split(r'[,/()\-\s]+', key):
        token_cap = token.capitalize()
        if token_cap in norway_cities:
            cache[name] = token_cap
            return token_cap

    # 5. Geopy lookup (only for promising candidates)
    if len(name) > 2 and any(char.isalpha() for char in name):
        try:
            loc = geocode(f"{name}, Norway", addressdetails=True, country_codes="no")
            if loc and (addr := loc.raw.get("address")):
                for k in CITY_KEYS:
                    if k in addr:
                        cache[name] = addr[k]
                        return addr[k]
        except Exception:
            pass

    # 6. GeoNames fallback (only for very specific cases)
    if len(name) > 3 and name.count(' ') <= 2:  # Skip complex names
        try:
            qs = urllib.parse.urlencode({
                "q": name,
                "country": GEONAMES_COUNTRY,
                "maxRows": 1,
                "username": GEONAMES_USER
            })
            url = f"http://api.geonames.org/searchJSON?{qs}"
            with urllib.request.urlopen(url, timeout=3) as resp:  # Reduced timeout
                data = json.load(resp)
                if data.get("geonames"):
                    city = data["geonames"][0]["name"]
                    cache[name] = city
                    return city
        except Exception:
            pass

    # 7. Record failure
    cache[name] = None
    return None

# Load and process input
print("Loading input data...")
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    records = json.load(f)

# Get unique venue names and filter out already cached ones
unique_names = sorted({rec.get("venuename") for rec in records if rec.get("venuename")})
uncached_names = [name for name in unique_names if name not in cache]

print(f"Total unique venues: {len(unique_names)}")
print(f"Already cached: {len(unique_names) - len(uncached_names)}")
print(f"Need to process: {len(uncached_names)}")

Loading input data...
Total unique venues: 1302
Already cached: 1297
Need to process: 5


In [6]:
# Process only uncached names
start_time = time.time()
for idx, vn in enumerate(uncached_names, start=1):
    city = get_city_for(vn)
    elapsed = time.time() - start_time
    rate = idx / elapsed if elapsed > 0 else 0
    print(f"\r[{idx:4d}/{len(uncached_names):4d}] {vn[:30]:30s} → {city} ({rate:.1f}/s)", end="")
    
    # Save cache more frequently for long-running processes
    if idx % 50 == 0:
        with open(CACHE_PATH, "wb") as cf:
            pickle.dump(cache, cf)

print(f"\n Processing complete in {time.time() - start_time:.1f} seconds")

# Annotate and save
print("Annotating records and saving...")
for rec in records:
    vn = rec.get("venuename") or ""
    rec["venuecity"] = get_city_for(vn) or ""

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=2)
with open(CACHE_PATH, "wb") as cf:
    pickle.dump(cache, cf)

# Stats
filled = sum(1 for r in records if r.get("venuecity") and r["venuecity"].strip())
total_records = len(records)
print(f"venuecity filled for {filled} of {total_records} records ({(filled / total_records) * 100:.2f}%)")

[   5/   5] Vigelandsstua ved Lindesnes un → None (0.0/s)0/s)
 Processing complete in 0.0 seconds
Annotating records and saving...


venuecity filled for 3245 of 4924 records (65.90%)


The script above pre-processes only those unique venue names for geocoding, and later all 4900+ records are annotated using the chached lookups.

### Mapping Works and Venues to authoritative IDs

The next step is to map every key `workid` and `venueid` with an authoritative URI, to resolve external Wikidata URIs for works/venue. To achieve this result, the optimal way is to keep the original keys for `workid` and `venueid` from IbsenStage and add the keys `workURI` and `venueURI` to the json. First I will map the works and then the venues.

A SPARQL query is sent to the Wikidata endpoint to retrieve all known works (`wdt:P800`) attributed to Henrik Ibsen (`wd:Q36661`). Labels are filtered to include multiple languages (en, no, nb, nn) and normalized to lowercase for matching. The script then attempts to match each worktitle from the dataset to a Wikidata label in two passes:

1. Exact match based on normalized title.

2. Partial string match (i.e. “Gjengangere” may match “Ghosts” or “The Ghosts”).

If a match is found, a new field workURI is added with the corresponding Wikidata QID 

In [7]:
# Load the dataset
with open('IbsenStage_with_city.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Query Wikidata for Henrik Ibsen's notable works, direct SPARQL query
def get_ibsen_works():
    sparql_query = """
    SELECT ?work ?label WHERE {
      wd:Q36661 wdt:P800 ?work .
      ?work rdfs:label ?label .
      FILTER(LANG(?label) IN ("en", "no", "nb", "nn"))
    }
    """
    url = "https://query.wikidata.org/sparql"
    headers = {'User-Agent': 'IbsenStage-Pipeline/1.0'}
    params = {'query': sparql_query, 'format': 'json'}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"SPARQL query failed with status code {response.status_code}")

print("Fetching Ibsen works from Wikidata...")
wikidata_result = get_ibsen_works()

# Preview first 5 entries
print("First 5 results from Wikidata:")
for item in wikidata_result['results']['bindings'][:5]:
    print(item)

Fetching Ibsen works from Wikidata...
First 5 results from Wikidata:
{'work': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q176465'}, 'label': {'xml:lang': 'en', 'type': 'literal', 'value': 'Hedda Gabler'}}
{'work': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q176465'}, 'label': {'xml:lang': 'nb', 'type': 'literal', 'value': 'Hedda Gabler'}}
{'work': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q176465'}, 'label': {'xml:lang': 'nn', 'type': 'literal', 'value': 'Hedda Gabler'}}
{'work': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q208094'}, 'label': {'xml:lang': 'en', 'type': 'literal', 'value': 'Peer Gynt'}}
{'work': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q208094'}, 'label': {'xml:lang': 'nb', 'type': 'literal', 'value': 'Peer Gynt'}}


In [8]:
# Build title → QID mapping from SPARQL result
wikidata_mapping = {}
for item in wikidata_result['results']['bindings']:
    uri = item['work']['value']
    qid = uri.split('/')[-1]  # Extract QID from URI
    label = item['label']['value'].lower().strip()
    wikidata_mapping[label] = qid

print(f"Found {len(wikidata_mapping)} labels from Wikidata")

# Show first 5 items in the mapping
print("First 5 label → QID mappings:")
for label, qid in list(wikidata_mapping.items())[:5]:
    print(f"{label} → {qid}")

Found 54 labels from Wikidata
First 5 label → QID mappings:
hedda gabler → Q176465
peer gynt → Q208094
emperor and galilean → Q268276
little eyolf → Q983970
lille eyolf → Q983970


In [9]:
# Function to map title to QID
def map_to_qid(title):
    if pd.isna(title) or not title.strip():
        return None
    title_norm = title.lower().strip()
    
    # Direct match
    if title_norm in wikidata_mapping:
        return wikidata_mapping[title_norm]
    
    # Partial match (fuzzy matching)
    for label, qid in wikidata_mapping.items():
        if title_norm in label or label in title_norm:
            return qid
    
    return None

# Apply mapping to dataset
print("Mapping 'worktitle' to Wikidata QIDs...")
df['workURI'] = df['worktitle'].apply(map_to_qid)

# Save to file
output_path = 'IbsenStage_with_wikidata_works.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(df.to_dict(orient='records'), f, ensure_ascii=False, indent=2)

print(f"Saved updated file to: {output_path}")

# Statistics
mapped_count = df['workURI'].notna().sum()
total_count = len(df)
print(f"Successfully mapped {mapped_count} out of {total_count} work titles ({mapped_count/total_count*100:.1f}%)")

Mapping 'worktitle' to Wikidata QIDs...
Saved updated file to: IbsenStage_with_wikidata_works.json
Successfully mapped 4905 out of 4924 work titles (99.6%)


Now I can proceed by connecting `venueURI` to authoritative identifiers from Wikidata, where available.

As expected, retrieving and mapping Wikidata URIs for all venues is a challenging process. Many venues are too small or obscure to have their own Wikidata item, or they may be part of a larger building. Some may no longer exist, as the IbsenStage dataset includes historical venues dating back to the 19th century.
To address this, I use a two-step fallback strategy:

1. Attempt to resolve the venuename to a `venueURI` by querying the Wikidata Search API (wbsearchentities).

2. If no match is found, attempt to resolve the `venuecity` instead and store the resulting URI under the key `cityURI`.

This method ensures that even if a venue does not have its own Wikidata item, the city associated with it can serve as a proxy reference.
To improve performance, the code uses a thread pool (`ThreadPoolExecutor`); `aiohttp` for asynchronous HTTP requests and `asyncio.gather()` to run all tasks concurrently (with a semaphore to limit concurrency to `MAX_WORKERS`). Despite this optimization, the cell may still take several minutes to complete (around 7) due to the delay between requests and the need to wait for responses from the Wikidata servers.

In [ ]:
INPUT = "IbsenStage_with_wikidata_works.json"
OUTPUT = "IbsenStage_with_uris.json"
USER_AGENT = "VenueCityWikidataLinker/1.0"
MAX_WORKERS = 5
DELAY = 0.1

# logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger()

# file path
def stage_path(filename):
    return Path.cwd() / filename

# loading data
with open(INPUT, encoding="utf-8") as f:
    data = json.load(f)

# print samples
print("CONFIGURATION AND DATA LOADING COMPLETE")
print(f"Input file: {INPUT}")
print(f"Output file: {OUTPUT}")
print(f"User agent: {USER_AGENT}")
print(f"Max concurrent workers: {MAX_WORKERS}")
print(f"Delay between requests: {DELAY}s")
print(f"Total records loaded: {len(data)}")
print("\nSample records:")
for i, record in enumerate(data[:3]):
    venue = record.get("venuename", "N/A")
    city = record.get("venuecity", "N/A")
    print(f"  Record {i+1}: Venue='{venue}', City='{city}'")

CONFIGURATION AND DATA LOADING COMPLETE
Input file: IbsenStage_with_wikidata_works.json
Output file: IbsenStage_with_uris.json
User agent: VenueCityWikidataLinker/1.0
Max concurrent workers: 5
Delay between requests: 0.1s
Total records loaded: 4924

Sample records:
  Record 1: Venue='Honningsvåg kino', City=''
  Record 2: Venue='Vadsø kino', City='Vadsø'
  Record 3: Venue='Miljøbygget', City='Trondheim'


In [12]:
# wikidata lookup helpers
async def query_wikidata(session, search_term, semaphore):
    if not search_term:
        return None
    async with semaphore:
        await asyncio.sleep(DELAY)
        params = urllib.parse.urlencode({
            "action": "wbsearchentities",
            "format": "json",
            "language": "en",
            "search": search_term,
            "limit": 1,
            "type": "item"
        })
        url = f"https://www.wikidata.org/w/api.php?{params}"
        headers = {"User-Agent": USER_AGENT}
        try:
            async with session.get(url, headers=headers, timeout=10) as response:
                result = await response.json()
                if result.get("search"):
                    return result["search"][0]["id"]
        except Exception as e:
            logger.debug(f"Wikidata query failed for '{search_term}': {e}")
        return None

async def resolve_entry(session, entry, semaphore):
    venue = (entry.get("venuename") or "").strip()
    city = (entry.get("venuecity") or "").strip()
    venue_uri = await query_wikidata(session, venue, semaphore)
    if venue_uri:
        entry["venueURI"] = venue_uri
    else:
        city_uri = await query_wikidata(session, city, semaphore)
        if city_uri:
            entry["cityURI"] = city_uri
    return entry

async def resolve_all(entries):
    semaphore = asyncio.Semaphore(MAX_WORKERS)
    async with aiohttp.ClientSession() as session:
        tasks = [resolve_entry(session, entry, semaphore) for entry in entries]
        return await asyncio.gather(*tasks)

# resolution with await
print("\nSTARTING WIKIDATA URI RESOLUTION")
start_time = time.time()
results = await resolve_all(data)
processing_time = time.time() - start_time
print(f"Processing completed in {processing_time:.2f} seconds")

# stats
venue_uris = sum(1 for r in results if r.get("venueURI"))
city_uris = sum(1 for r in results if r.get("cityURI"))
total_uris = venue_uris + city_uris

print("\nPROCESSING RESULTS")
print(f"Venue URIs found: {venue_uris}")
print(f"City URIs found: {city_uris}")
print(f"Total URIs resolved: {total_uris} out of {len(results)} records ({(total_uris/len(results)*100):.1f}%)")

print("\nSample results with URIs:")
uri_samples = [r for r in results if r.get("venueURI") or r.get("cityURI")][:3]
for i, record in enumerate(uri_samples):
    venue = record.get("venuename", "N/A")
    city = record.get("venuecity", "N/A")
    venue_uri = record.get("venueURI", "N/A")
    city_uri = record.get("cityURI", "N/A")
    print(f"  Sample {i+1}: Venue='{venue}' (URI: {venue_uri}), City='{city}' (URI: {city_uri})")

# saving output
with open(stage_path(OUTPUT), "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\nMapping complete. Output saved to: {OUTPUT}")


STARTING WIKIDATA URI RESOLUTION
Processing completed in 347.20 seconds

=== PROCESSING RESULTS ===
Venue URIs found: 2883
City URIs found: 743
Total URIs resolved: 3626 out of 4924 records (73.6%)

Sample results with URIs:
  Sample 1: Venue='Vadsø kino' (URI: N/A), City='Vadsø' (URI: Q104379)
  Sample 2: Venue='Miljøbygget' (URI: N/A), City='Trondheim' (URI: Q25804)
  Sample 3: Venue='Telemark Teater' (URI: Q4453420), City='' (URI: N/A)

Mapping complete. Output saved to: IbsenStage_with_uris.json


Now that both work and venue IDs and URIs are present, we can bridge them in a single file.

In [ ]:
# Load the enriched data
with open('Ibsenstage_staged/IbsenStage_with_uris.json', encoding='utf-8') as f:
    data = json.load(f)
df = pd.DataFrame(data)

# Create bridge mappings
work_bridge = (
    df[['workid', 'workURI']]
    .dropna()
    .drop_duplicates()
    .set_index('workid')['workURI']
    .to_dict()
)

venue_bridge = (
    df[['venueid', 'venueURI']]
    .dropna()
    .drop_duplicates()
    .set_index('venueid')['venueURI']
    .to_dict()
)

# Combine into one object
bridge = {
    'work_bridge': work_bridge,
    'venue_bridge': venue_bridge
}

# Ensure staged directory exists
staged_dir = Path.cwd() / 'Ibsenstage_staged'
staged_dir.mkdir(exist_ok=True)

# Save the mapping
output_path = staged_dir / 'id_to_uri_bridge.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(bridge, f, ensure_ascii=False, indent=2)

print(f"ID-to-URI bridge saved to {output_path}")

✅ ID-to-URI bridge saved to c:\Users\Cristiano (CC)\Desktop\Cristiano-June25\OsloMet\Masterstudium i bibliotek- og informasjonsvitenskap - deltid\MBIB4140 - Metadata og interoperabilitet\2ndre sjansen\Ibsenstage_staged\id_to_uri_bridge.json
